In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sqlalchemy import create_engine

In [2]:
imdb_df = pd.read_csv("Resources/IMDB-Movie-Data.csv")
streaming_df = pd.read_csv("Resources/MoviesOnStreamingPlatforms_updated.csv")

In [3]:
imdb_df



,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0


In [4]:
year_title=imdb_df["Year"].astype(str) + "_" + imdb_df["Title"]
imdb_df["year_title"] =  year_title
imdb_df

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,year_title
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,2014_Guardians of the Galaxy
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,2012_Prometheus
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,2016_Split
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,2016_Sing
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,2016_Suicide Squad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0,2015_Secret in Their Eyes
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0,2007_Hostel: Part II
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0,2008_Step Up 2: The Streets
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0,2014_Search Party


In [5]:
#imdb_split_list = imdb_df.Actors.str.split(',').tolist()

imdb_stack_df = pd.DataFrame(imdb_df.Actors.str.split(',').tolist(), index=imdb_df.year_title).stack().reset_index()
imdb_stack_df

,year_title,level_1,0
0,2014_Guardians of the Galaxy,0,Chris Pratt
1,2014_Guardians of the Galaxy,1,Vin Diesel
2,2014_Guardians of the Galaxy,2,Bradley Cooper
3,2014_Guardians of the Galaxy,3,Zoe Saldana
4,2012_Prometheus,0,Noomi Rapace
...,...,...,...
3994,2014_Search Party,3,Shannon Woodward
3995,2016_Nine Lives,0,Kevin Spacey
3996,2016_Nine Lives,1,Jennifer Garner
3997,2016_Nine Lives,2,Robbie Amell


In [6]:
imdb_stack_df = imdb_stack_df.drop(columns="level_1")
imdb_stack_df

,year_title,0
0,2014_Guardians of the Galaxy,Chris Pratt
1,2014_Guardians of the Galaxy,Vin Diesel
2,2014_Guardians of the Galaxy,Bradley Cooper
3,2014_Guardians of the Galaxy,Zoe Saldana
4,2012_Prometheus,Noomi Rapace
...,...,...
3994,2014_Search Party,Shannon Woodward
3995,2016_Nine Lives,Kevin Spacey
3996,2016_Nine Lives,Jennifer Garner
3997,2016_Nine Lives,Robbie Amell


In [7]:
imdb_stack_df

,year_title,0
0,2014_Guardians of the Galaxy,Chris Pratt
1,2014_Guardians of the Galaxy,Vin Diesel
2,2014_Guardians of the Galaxy,Bradley Cooper
3,2014_Guardians of the Galaxy,Zoe Saldana
4,2012_Prometheus,Noomi Rapace
...,...,...
3994,2014_Search Party,Shannon Woodward
3995,2016_Nine Lives,Kevin Spacey
3996,2016_Nine Lives,Jennifer Garner
3997,2016_Nine Lives,Robbie Amell


In [8]:
imdb_stack_df.columns = ["year_title", "Actor"]
imdb_stack_df

,year_title,Actor
0,2014_Guardians of the Galaxy,Chris Pratt
1,2014_Guardians of the Galaxy,Vin Diesel
2,2014_Guardians of the Galaxy,Bradley Cooper
3,2014_Guardians of the Galaxy,Zoe Saldana
4,2012_Prometheus,Noomi Rapace
...,...,...
3994,2014_Search Party,Shannon Woodward
3995,2016_Nine Lives,Kevin Spacey
3996,2016_Nine Lives,Jennifer Garner
3997,2016_Nine Lives,Robbie Amell


In [29]:
# This cell needs work

actors = imdb_stack_df.Actor.unique()
actors_df = pd.DataFrame(actors).sort_values(by=acto"Actor")
actors_df

KeyError: 'Actor'

In [12]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  872 non-null    float64
 11  Metascore           936 non-null    float64
 12  year_title          1000 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 101.7+ KB


In [10]:
streaming_df

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16739,16739,16740,The Ghosts of Buxley Hall,1980,NaN,6.2,NaN,0,0,0,1,0,Bruce Bilson,"Comedy,Family,Fantasy,Horror",United States,English,120.0
16740,16740,16741,The Poof Point,2001,7+,4.7,NaN,0,0,0,1,0,Neal Israel,"Comedy,Family,Sci-Fi",United States,English,90.0
16741,16741,16742,Sharks of Lost Island,2013,NaN,5.7,NaN,0,0,0,1,0,Neil Gelinas,Documentary,United States,English,NaN
16742,16742,16743,Man Among Cheetahs,2017,NaN,6.6,NaN,0,0,0,1,0,Richard Slater-Jones,Documentary,United States,English,NaN


In [13]:
streaming_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       16744 non-null  int64  
 1   ID               16744 non-null  int64  
 2   Title            16744 non-null  object 
 3   Year             16744 non-null  int64  
 4   Age              7354 non-null   object 
 5   IMDb             16173 non-null  float64
 6   Rotten Tomatoes  5158 non-null   object 
 7   Netflix          16744 non-null  int64  
 8   Hulu             16744 non-null  int64  
 9   Prime Video      16744 non-null  int64  
 10  Disney+          16744 non-null  int64  
 11  Type             16744 non-null  int64  
 12  Directors        16018 non-null  object 
 13  Genres           16469 non-null  object 
 14  Country          16309 non-null  object 
 15  Language         16145 non-null  object 
 16  Runtime          16152 non-null  float64
dtypes: float64(2

In [14]:
year_title=streaming_df["Year"].astype(str) + "_" + streaming_df["Title"]
streaming_df["year_title"] =  year_title
streaming_df

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,year_title
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,2010_Inception
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,1999_The Matrix
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,2018_Avengers: Infinity War
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,1985_Back to the Future
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,"1966_The Good, the Bad and the Ugly"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16739,16739,16740,The Ghosts of Buxley Hall,1980,NaN,6.2,NaN,0,0,0,1,0,Bruce Bilson,"Comedy,Family,Fantasy,Horror",United States,English,120.0,1980_The Ghosts of Buxley Hall
16740,16740,16741,The Poof Point,2001,7+,4.7,NaN,0,0,0,1,0,Neal Israel,"Comedy,Family,Sci-Fi",United States,English,90.0,2001_The Poof Point
16741,16741,16742,Sharks of Lost Island,2013,NaN,5.7,NaN,0,0,0,1,0,Neil Gelinas,Documentary,United States,English,NaN,2013_Sharks of Lost Island
16742,16742,16743,Man Among Cheetahs,2017,NaN,6.6,NaN,0,0,0,1,0,Richard Slater-Jones,Documentary,United States,English,NaN,2017_Man Among Cheetahs


In [17]:
merge_df = pd.merge(imdb_df, streaming_df, how="inner", on="year_title")

In [18]:
merge_df

,Rank,Title_x,Genre,Description,Director,Actors,Year_x,Runtime (Minutes),Rating,Votes,...,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0,0,0,1,0,James Gunn,"Action,Adventure,Comedy,Sci-Fi",United States,English,121.0
1,8,Mindhorn,Comedy,A has-been actor best known for playing the ti...,Sean Foley,"Essie Davis, Andrea Riseborough, Julian Barrat...",2016,89,6.4,2490,...,1,0,0,0,0,Sean Foley,Comedy,United Kingdom,English,89.0
2,9,The Lost City of Z,"Action,Adventure,Biography","A true-life drama, centering on British explor...",James Gray,"Charlie Hunnam, Robert Pattinson, Sienna Mille...",2016,141,7.1,7188,...,0,0,1,0,0,James Gray,"Biography,Drama,History",United States,"English,Portuguese,Tupi,Spanish,German",141.0
3,14,Moana,"Animation,Adventure,Comedy","In Ancient Polynesia, when a terrible curse in...",Ron Clements,"Auli'i Cravalho, Dwayne Johnson, Rachel House,...",2016,107,7.7,118151,...,0,0,0,1,0,"Ron Clements,John Musker,Don Hall,Chris Williams","Animation,Adventure,Comedy,Family,Fantasy,Musical",United States,English,107.0
4,15,Colossal,"Action,Comedy,Drama",Gloria is an out-of-work party girl forced to ...,Nacho Vigalondo,"Anne Hathaway, Jason Sudeikis, Austin Stowell,...",2016,109,6.4,8612,...,0,1,0,0,0,Nacho Vigalondo,"Comedy,Drama,Fantasy,Sci-Fi,Thriller","Canada,United States,Spain,South Korea","English,Korean",109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,978,Amateur Night,Comedy,Guy Carter is an award-winning graduate studen...,Lisa Addario,"Jason Biggs, Janet Montgomery,Ashley Tisdale, ...",2016,92,5.0,2229,...,0,0,1,0,0,"Matt Bettinelli-Olpin,David Bruckner,Tyler Gil...","Horror,Thriller",United States,English,116.0
227,991,Underworld: Rise of the Lycans,"Action,Adventure,Fantasy",An origins story centered on the centuries-old...,Patrick Tatopoulos,"Rhona Mitra, Michael Sheen, Bill Nighy, Steven...",2009,92,6.6,129708,...,1,0,0,0,0,Patrick Tatopoulos,"Action,Fantasy","United States,New Zealand",English,92.0
228,994,Resident Evil: Afterlife,"Action,Adventure,Horror",While still out to destroy the evil Umbrella C...,Paul W.S. Anderson,"Milla Jovovich, Ali Larter, Wentworth Miller,K...",2010,97,5.9,140900,...,1,0,0,0,0,Paul W.S. Anderson,"Action,Adventure,Fantasy,Horror,Sci-Fi,Thriller","Germany,France,United States,Canada,United Kin...","English,Japanese,Spanish",96.0
229,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,1,0,0,0,0,Billy Ray,"Crime,Drama,Mystery,Thriller","United States,United Kingdom,Spain,South Korea",English,111.0


In [21]:
merge_left_df = pd.merge(imdb_df, streaming_df, how="left", on="year_title")

In [22]:
merge_left_df

,Rank,Title_x,Genre,Description,Director,Actors,Year_x,Runtime (Minutes),Rating,Votes,...,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0.0,0.0,0.0,1.0,0.0,James Gunn,"Action,Adventure,Comedy,Sci-Fi",United States,English,121.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,1.0,0.0,0.0,0.0,0.0,Billy Ray,"Crime,Drama,Mystery,Thriller","United States,United Kingdom,Spain,South Korea",English,111.0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,1.0,0.0,0.0,0.0,0.0,NaN,"Comedy,Crime,Drama,Mystery,Romance,Thriller",United States,English,24.0


In [ ]:
# Create a dataframe of just actors
actors_df = pd.DataFrame(imdb_stack_df ["Actor"])
actors_df

In [ ]:
# Remove duplicates so only one record for each actor
actors_df = actors_df.drop_duplicates()
actors_df

In [18]:
# Create a dataframe for just directors
directors_df = pd.DataFrame(imdb_df["Director"])
directors_df

,Director
0,James Gunn
1,Ridley Scott
2,M. Night Shyamalan
3,Christophe Lourdelet
4,David Ayer
...,...
995,Billy Ray
996,Eli Roth
997,Jon M. Chu
998,Scot Armstrong


In [19]:
# Remove duplicates so only one record for each director
directors_df = directors_df.drop_duplicates()
directors_df

,Director
0,James Gunn
1,Ridley Scott
2,M. Night Shyamalan
3,Christophe Lourdelet
4,David Ayer
...,...
990,Patrick Tatopoulos
991,Aamir Khan
994,Nima Nourizadeh
995,Billy Ray
